# Getting Embeddings

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
import torch.nn as nn
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

# The model is running on CPU, since it is already pre-trained and doesnt require GPU
device = torch.device('cpu') 
print('Running on device: {}'.format(device))

#Define MTCNN module
#Since MTCNN is a collection of neural nets and other code, 
#The device must be passed in the following way to enable copying of objects when needed internally.
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device
)
#Function takes 2 vectors 'a' and 'b'
#Returns the cosine similarity according to the definition of the dot product
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

#cos_sim returns real numbers,where negative numbers have different interpretations.
#So we use this function to return only positive values.
def cos(a,b):
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)

# Define Inception Resnet V1 module (GoogLe Net)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define a dataset and data loader
dataset = datasets.ImageFolder('Film/Test')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=lambda x: x[0])

#Perfom MTCNN facial detection
#Detects the face present in the image and prints the probablity of face detected in the image.
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

# Calculate the 512 face embeddings
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).cpu()

# Print distance matrix for classes.
#The embeddings are plotted in space and cosine distace is measured.
cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
for i in range(0,len(names)):
    emb=embeddings[i].unsqueeze(0)
    # The cosine similarity between the embeddings is given by 'dist'.
    dist =cos(embeddings[0],emb)  
        
dists = [[cos(e1,e2).item() for e2 in embeddings] for e1 in embeddings]
# The print statement below is
#Helpful for analysing the results and for determining the value of threshold.
print(pd.DataFrame(dists, columns=names, index=names)) 


Running on device: cpu


C:\Users\91951\anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Face detected with probability: 0.999283
Face detected with probability: 0.999999
Face detected with probability: 0.999958
Face detected with probability: 0.999304
Face detected with probability: 1.000000
Face detected with probability: 0.999999
Face detected with probability: 0.999880
Face detected with probability: 0.999999
Face detected with probability: 0.999950
Face detected with probability: 0.999954
Face detected with probability: 0.999999
Face detected with probability: 0.990267
Face detected with probability: 0.999950
Face detected with probability: 0.999908
Face detected with probability: 1.000000
Face detected with probability: 0.994620
Face detected with probability: 0.999949
Face detected with probability: 1.000000
Face detected with probability: 0.999930
Face detected with probability: 0.999945
                  Deepika Padukone  Deepika Padukone  Hritik Roshan  \
Deepika Padukone          1.000000          0.849726       0.611940   
Deepika Padukone          0.849726    

# Calculating Accuracy

In [4]:
import glob
import random
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1,extract_face
from PIL import Image,ImageDraw
import torch
import cv2
import torch.nn as nn

#Define list 'name' with the names of subjects in dataset.
name= ['Deepika Padukone', 'Hritik Roshan', 'Mammootty', 'Mohanlal', 'Nazriya', 'Nivin Pauly', 'ShahRukh Khan', 'Shobana', 'Sridevi Kapoor', 'Varun Dhawan']  #names of the folders in ur prediction dataset

#Model running on CPU
device = torch.device('cpu')  

#Define Inception V1 module (GoogLeNet)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to('cpu')

#Define MTCNN module
#Since MTCNN is a collection of neural nets and other code, 
#The device must be passed in the following way to enable copying of objects when needed internally.
#'keep_all' is kept True. All the faces present in the image will be detected.
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device,keep_all=True
)

#Function for performing face recognition
def verify(embedding):
    for i,k in enumerate(embeddings):
        for j,l in enumerate(embedding):
            #Cosine similarity distance is calculated.
            dist =cos(k,l)
            
            #Chosen threshold is 0.81
            #Threshold is determined after seeing the table in the previous cell.
            #If the cosine distance is more than 0.85, name of that person is returned.
            if dist > 0.81:
                return names[i]
            
#Function to get file list and save it as prediction dataset.
#Return prediction dataset.
def get_files(nam): 
    files = glob.glob("Predict/%s/*" %nam)
    prediction = files[:int(len(files))] 
    return prediction

#Function for storing and returning files and their corresponding labels in the prediction dataset.
def make_sets():  
    prediction_data = []
    prediction_labels = []
    
    # The face is cropped from image after detecting face using MTCNN.
    #prediction_data and prediction_labels are then updated.
    for nam in name:
        prediction = get_files(nam)
        for item in prediction: 
            img = Image.open(item)
            img_cropped = mtcnn(img)
            prediction_data.append(img_cropped)
            prediction_labels.append(nam)
     
    return prediction_data, prediction_labels

#Function that prints incorrect match and returns the percentage accuracy 
def run_recognizer():
    prediction_data, prediction_labels = make_sets()
    print ("Predicting classification set...")
    cnt = 0
    correct = 0
    incorrect = 0
    none=0
    for image in prediction_data:
        im=resnet(image)
        pred=verify(im)
        #If the prediction matched with the testing images, correct variable is incremented
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        #else, the incorrect match is printed on screen
        #incorrect varable is incremented.
        else:
            print(prediction_labels[cnt],' predicted as ',pred)
            incorrect += 1
            cnt += 1
    #Returning the perentage of how many are predicted correctly.
    return ((100*correct)/(correct + incorrect))

metascore = []
print("\n")
for i in range(0,1):
    correct = run_recognizer()
    #metascore is updated, each time the loop runs
    metascore.append(correct)
    
#Finally, the mean percentage is calcuted and printed on screen.
print("\nDataset accuracy: ", np.mean(metascore), "% correct!!")



Predicting classification set...
Hritik Roshan  predicted as  None
Nazriya  predicted as  None
Nivin Pauly  predicted as  Mammootty
Shobana  predicted as  None

Dataset accuracy:  96.0 % correct!!
